<a href="https://colab.research.google.com/github/anantasty/qlora-llama2/blob/main/Copy_of_qlora_llama2_clean_PubMed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch accelerate bitsandbytes datasets transformers peft trl scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 9.5 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sun Nov 26 23:43:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers

import torch
import torch.nn as nn

***Get Model***

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
#model_id = 'meta-llama/Llama-2-13b-chat-hf'
model_id = 'NousResearch/Llama-2-7b-chat-hf' # "meta-llama/Llama-2-7b-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


In [10]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=True
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

In [11]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=True
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Model loaded on cuda:0


In [12]:
mem = model.get_memory_footprint()
print("Memory footprint: {} ".format(mem))

# should be (7B) 7,000,000,000*4(Int4) / 8(8 bits per byte) = 3,500,000,000 = 3.5GB
# actual (7B)  3,829,940,224 (not all weights become int 4)
# actual (13B)  7,083,970,560  (not all weights become int 4)

Memory footprint: 3829940224 


In [13]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

***Get Dataset***

In [14]:
# Load the dataset from Hugging Face
from datasets import load_dataset

dataset = load_dataset("pubmed_qa", "pqa_labeled", split="train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
dataset = dataset.filter(lambda example: example['final_decision'] == "yes")

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
print(f'Number of records: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

'''
Number of records: 1837928
Column names are: ['source', 'target', 'rationale', 'task', 'type']
'''

Number of records: 552
Column names are: ['pubid', 'question', 'context', 'long_answer', 'final_decision']


"\nNumber of records: 1837928\nColumn names are: ['source', 'target', 'rationale', 'task', 'type']\n"

In [17]:
def create_prompt(rec):
    contactenated_context = "".join(rec['context']["contexts"])

    start = "Answer the below question based on the context provided."
    question = f"### QUESTION:\n{rec['question']}\n\n"
    context = f"Context:\n {contactenated_context}"
    response = f"### ANSWER:\n{rec['long_answer']}\n\n"
    end = "### End"

    parts = [part for part in [start, question, context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    formatted_prompt = formatted_prompt.replace('\\n', '\n')

    rec["text"] = formatted_prompt

    return rec

In [20]:
p = create_prompt(dataset[300])
print(p)
print(p["text"])

{'pubid': 12632437, 'question': 'Are environmental factors important in primary systemic vasculitis?', 'context': {'contexts': ['To investigate the association between primary systemic vasculitis (PSV) and environmental risk factors.', "Seventy-five PSV cases and 273 controls (220 nonvasculitis, 19 secondary vasculitis, and 34 asthma controls) were interviewed using a structured questionnaire. Factors investigated were social class, occupational and residential history, smoking, pets, allergies, vaccinations, medications, hepatitis, tuberculosis, and farm exposure in the year before symptom onset (index year). The Standard Occupational Classification 2000 and job-exposure matrices were used to assess occupational silica, solvent, and metal exposure. Stepwise multiple logistic regression was used to calculate the odds ratio (OR) and 95% confidence interval (95% CI) adjusted for potential confounders. Total PSV, subgroups (47 Wegener's granulomatosis [WG], 12 microscopic polyangiitis, 16

In [22]:
dataset = dataset.map(create_prompt)

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [24]:
dataset = dataset.map(
        batched=True,
        remove_columns=['pubid', 'question', 'context', 'long_answer', 'final_decision']
    )

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [27]:
print(dataset[307]["text"])

Answer the below question based on the context provided.

### QUESTION:
Acute hepatitis C in Israel: a predominantly iatrogenic disease?



Context:
 Acute hepatitis C virus infection in the era of universal screening of blood products has not disappeared, and is thought to be transmitted primarily via injecting drug use. A growing body of evidence supports iatrogenic transmission as an important mode of transmission. The aim of this study was to examine transmission routes and clinical characteristics in a group of patients with acute hepatitis C in Israel.A retrospective chart review was conducted in three different liver clinics in Israel, of all new hepatitis C patients. Patients identified as possible acute hepatitis C were re-interviewed and all other sources such as blood bank records and pre-employment check-ups reviewed in order to establish the diagnosis of acute hepatitis C infection and to identify the transmission route.Twenty-nine patients were found to have acute hepatit

In [ ]:
#Save dataset to the hub for future use
#dataset.push_to_hub("Venkat-Ram-Rao/processed_cot_dataset", private=True)

In [28]:
#max length of the model
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [29]:
mx = get_max_length(model)
mx

Found max lenth: 4096


4096

In [30]:
len(dataset)

552

In [31]:
#tokenize dataset
dataset = dataset.map(lambda samples: tokenizer(samples['text']), batched=True)

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [32]:
len(dataset)

552

In [33]:
dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < mx)

Filter:   0%|          | 0/552 [00:00<?, ? examples/s]

In [ ]:
len(dataset)

In [35]:
seed = 42
set_seed(seed)

dataset = dataset.shuffle(seed=seed)

***Freeze Original Weights***

In [36]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

***Create Lora Config***

In [37]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [38]:
modules = find_all_linear_names(model)
print(modules)

#['v_proj', 'up_proj', 'down_proj', 'k_proj', 'o_proj', 'q_proj', 'gate_proj']

['q_proj', 'gate_proj', 'k_proj', 'down_proj', 'v_proj', 'o_proj', 'up_proj']


In [39]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,  #attention heads
    lora_alpha=64,  #alpha scaling
    target_modules=modules,  #gonna train all
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM", #for Decoder models like GPT Seq2Seq for Encoder-Decoder models like T5
)



In [40]:
##Get the PEFT Model using the downloaded model and the loRA config
model = get_peft_model(model, config)

***Training***

In [41]:
# Print Trainable parameters
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(
    f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


In [42]:
tokenizer.pad_token = tokenizer.eos_token

In [43]:
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100, #20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.071400
2,1.822300
3,1.795900
4,1.733300
5,2.016800
6,1.548200
7,1.806500
8,1.667000
9,1.658800
10,1.553800


TrainOutput(global_step=100, training_loss=1.4355390107631683, metrics={'train_runtime': 1252.9193, 'train_samples_per_second': 0.319, 'train_steps_per_second': 0.08, 'total_flos': 7957959479328768.0, 'train_loss': 1.4355390107631683, 'epoch': 0.72})

***Push to Hub***

In [ ]:
model.push_to_hub("Venkat-Ram-Rao/Llama2_7B_qlora_CoT_FT-v2",
                  use_auth_token=True,
                  commit_message="fine tuned on kaist-ai/CoT-Collection",
                  private=True)

***Load from Hub***

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Venkat-Ram-Rao/Llama2_7B_qlora_CoT_FT"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [44]:
mem = model.get_memory_footprint()
print("Memory footprint: {} ".format(mem))

#7,227,846,656

Memory footprint: 3990380544 


***Inference***

In [47]:
tst = """Answer the below question based on the context provided.

### QUESTION:
What can Product ibuprofen be used to treat?



Context:
What is ibuprofen?
Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by reducing hormones that cause inflammation and pain in the body.

Ibuprofen is used to reduce fever and treat pain or inflammation caused by many conditions such as headache, toothache, back pain, arthritis, menstrual cramps, or minor injury.

Ibuprofen is used in adults and children who are at least 6 months old.

Warnings
Ibuprofen can increase your risk of fatal heart attack or stroke. Do not use this medicine just before or after heart bypass surgery (coronary artery bypass graft, or CABG).

Ibuprofen may also cause stomach or intestinal bleeding, which can be fatal. These conditions can occur without warning while you are using this medicine, especially in older adults.

Do not take more than your recommended dose. An ibuprofen overdose can damage your stomach or intestines. Use only the smallest amount of medication needed to get relief from your pain, swelling, or fever.




### ANSWER:"""

In [48]:
batch = tokenizer(tst, return_tensors='pt')



In [49]:
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

'''
EXPECTED ANSWER:

### RESPONSE:
The given list ['i', 'P', 'h', '849', 'e'] contains the number 849, which is the only element of the list that is a number. Therefore, the final answer is 849.

Therefore the answer is 849

### End'''

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.p



 Answer the below question based on the context provided.

### QUESTION:
What can Product ibuprofen be used to treat?



Context:
What is ibuprofen?
Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by reducing hormones that cause inflammation and pain in the body.

Ibuprofen is used to reduce fever and treat pain or inflammation caused by many conditions such as headache, toothache, back pain, arthritis, menstrual cramps, or minor injury.

Ibuprofen is used in adults and children who are at least 6 months old.

Warnings
Ibuprofen can increase your risk of fatal heart attack or stroke. Do not use this medicine just before or after heart bypass surgery (coronary artery bypass graft, or CABG).

Ibuprofen may also cause stomach or intestinal bleeding, which can be fatal. These conditions can occur without warning while you are using this medicine, especially in older adults.

Do not take more than your recommended dose. An ibuprofen overdose can damage your stomach or

"\nEXPECTED ANSWER:\n\n### RESPONSE:\nThe given list ['i', 'P', 'h', '849', 'e'] contains the number 849, which is the only element of the list that is a number. Therefore, the final answer is 849.\n\nTherefore the answer is 849\n\n### End"

Test2

In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Sam is taller than Bob. Jack is taller than Sam. Is Jack taller than Bob?
Answer Yes or No



### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Read the Instruction below and provide an answer.

### INSTRUCTION:
Sam is taller than Bob. Jack is taller than Sam. Is Jack taller than Bob?
Answer Yes or No



### RESPONSE:
Sam is taller than Bob, and Jack is taller than Sam. Therefore, Jack is taller than Bob.


Therefore the answer is Yes



### End


Therefore the answer is Yes



### End of Inline Response



### End of Inline Response


Therefore the answer is Yes



### End of chat


In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Sam is taller than Bob. Jack is taller than Sam.
Who is the tallest? Bob, Sam or Jack?




### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', '100', 'h', '849', 'e']



### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', '100', 'h', '849', 'e']



### RESPONSE:
The elements of the list that are numbers are '100' and '849'. The sum of these two numbers is 949.


Therefore the answer is 949



### End


Therefore the answer is 949



### End of task



### End of task


Therefore the answer is


In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', '100', 'h', '849', '100']



### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you need to provide the correct option for a given problem on probability from the provided options. Problem: there is a 50 % chance jen will visit chile this year , while there is a 50 % chance that she will visit madagascar this year . what is the probability that jen will visit either chile or madagascar this year , but not both ?
Options: a ) 25.0 % , b ) 50.0 % , c ) 62.5 % , d ) 75.0 % , e ) 80.0 %



### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task you will be given a list of numbers and you need to find the mean (average) of that list. The mean of a list can be found by summing every number in the list then dividing the result by the size of that list. The output should be rounded to 3 decimal places.

 [-43.959, 161.939]



### RESPONSE:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs fo



 Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task you will be given a list of numbers and you need to find the mean (average) of that list. The mean of a list can be found by summing every number in the list then dividing the result by the size of that list. The output should be rounded to 3 decimal places.

 [-43.959, 161.939]



### RESPONSE:
The mean of the given list is 85.92.


Therefore the answer is 85.92



### End


Therefore the answer is 85.92



### End of task



### End of task


Therefore the answer is 85.92



### End of
